# Positionnement dans mon répertoire de travail (google drive)

In [1]:
pwd()

"D:\\Users\\Administrator\\OneDrive\\Documents\\Julia\\FEM"

# Se positionner dans son répertoire de téléchargement

In [2]:
;cd "G:\\My Drive\\"

G:\My Drive


# Instantiation de mon projet (téléchargement des dépendances)

In [2]:
]instantiate

 Activating environment at `G:\My Drive\_MOPJ\Julia\FEM\Project.toml`


# Listing des paquets disponibles dans l'environnement actuel 

In [3]:
]st

Status `G:\My Drive\_MOPJ\Julia\FEM\Project.toml`
  [3625dbae] LibFEM v0.1.0 `https://github.com/amdeld/LibFEM.jl#master`


# Si le résultat montre que le paquet LibFEM n'est pas présent, il faut alors  rajouter la librairie (paquet) pour faire du calcul de structures élastiques 0D/1D.
Téléchargement de la bibliothèque LibFEM depuis mon compte github https://github.com/amdeld

In [4]:
using Pkg;Pkg.add(url="https://github.com/amdeld/LibFEM.jl")
#]add https://github.com/amdeld/LibFEM.jl

   Updating git-repo `https://github.com/amdeld/LibFEM.jl`


   Updating registry at `C:\Users\Admin\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...
No Changes to `G:\My Drive\_MOPJ\Julia\FEM\Project.toml`
No Changes to `G:\My Drive\_MOPJ\Julia\FEM\Manifest.toml`


# On charge ensuite la librairie LibFEM avec la commande suivante:

In [5]:
using LibFEM # ,Plots

# Quelques informations quant au cas à traiter

In [6]:
#PARAMETERS
const L=10000. #length in mm
const A=100. #cross-sectional area in mm^2
const E=210000. #modulus of elasticity in MPa [steel]
const FM=10000.; #force modulus in N

In [7]:
# ===============================================PRE-PROCESSING==================
# DEFINING AND DISCRETIZING[MESHING] THE STRUCTURE
# connectivity table
# elt||node_i||node_j
# 1|1|3
# 2|2|3
# grid()
X1pos=0.;Y1pos=0.
X2pos=0.;Y2pos=L
X3pos=L;Y3pos=L
#lengths
L1=D2_TrussElementLength(X1pos,Y1pos,X3pos,Y3pos) #length of element 1
L2=D2_TrussElementLength(X2pos,Y2pos,X3pos,Y3pos) #length of element 2
#APPLYING GEOMETRIC&MATERIAL PROPERTIES
A1=sqrt(2)*A; #cross-sectional area of element 1
A2=A; #cross-sectional area of element 2
E1=E; #material of element 1
E2=E; #material of element 2

# Calcul et Affichage des matrices de raideur élémentaires

In [8]:
#writing-defining the element stiffness matrices
K1=D2_TrussElementStiffness(E1,A1,L1,45);
println("\nK1=\r")
K1


K1=


4×4 Array{Float64,2}:
  1050.0   1050.0  -1050.0  -1050.0
  1050.0   1050.0  -1050.0  -1050.0
 -1050.0  -1050.0   1050.0   1050.0
 -1050.0  -1050.0   1050.0   1050.0

In [9]:
K2=D2_TrussElementStiffness(E2,A2,L2,0);
println("\nK2=\r")
K2


K2=


4×4 Array{Float64,2}:
  2100.0   0.0  -2100.0  -0.0
     0.0   0.0     -0.0  -0.0
 -2100.0  -0.0   2100.0   0.0
    -0.0  -0.0      0.0   0.0

# Affichage des matrices de raideurs positionnées [K1]^p [K2]p et et de la matrice d'assemblage [K]

In [10]:
#ASSEMBLING THE GLOBAL STIFFNESS MATRIX
#matrices initialization
K=zeros(6,6);K1P=zeros(6,6);
#positionning stiffness matrices
K1P=D2_TrussAssemble(K,K1,1,3)
println("\nK1P=\r")
K1P


K1P=


6×6 Array{Float64,2}:
  1050.0   1050.0  0.0  0.0  -1050.0  -1050.0
  1050.0   1050.0  0.0  0.0  -1050.0  -1050.0
     0.0      0.0  0.0  0.0      0.0      0.0
     0.0      0.0  0.0  0.0      0.0      0.0
 -1050.0  -1050.0  0.0  0.0   1050.0   1050.0
 -1050.0  -1050.0  0.0  0.0   1050.0   1050.0

In [11]:
K=zeros(6,6);K2P=zeros(6,6);
K2P=D2_TrussAssemble(K,K2,2,3)
println("\nK2P=\r")
K2P


K2P=


6×6 Array{Float64,2}:
 0.0  0.0      0.0  0.0      0.0  0.0
 0.0  0.0      0.0  0.0      0.0  0.0
 0.0  0.0   2100.0  0.0  -2100.0  0.0
 0.0  0.0      0.0  0.0      0.0  0.0
 0.0  0.0  -2100.0  0.0   2100.0  0.0
 0.0  0.0      0.0  0.0      0.0  0.0

In [12]:
#assembling
K=K1P+K2P
println("\nK=\r")
K


K=


6×6 Array{Float64,2}:
  1050.0   1050.0      0.0  0.0  -1050.0  -1050.0
  1050.0   1050.0      0.0  0.0  -1050.0  -1050.0
     0.0      0.0   2100.0  0.0  -2100.0      0.0
     0.0      0.0      0.0  0.0      0.0      0.0
 -1050.0  -1050.0  -2100.0  0.0   3150.0   1050.0
 -1050.0  -1050.0      0.0  0.0   1050.0   1050.0

# Affichage des vecteurs des déplacements et des chargements nodaux

In [13]:
#SOLVING DISPACEMENT EQUATIONS
#extracting displacement submatrix via index vector
K_s=K[5:6,5:6]
#Setting-up the force subvector by applying Load & Boundary Conditions[LBC]]
F_s=[0 FM]'
#solving by gaussian elimination
U_s=K_s\F_s
#SOLVING FORCE EQUATIONS
#setting-up the global nodal displacement vector
U=[0 0 0 0 U_s[1:2]']'
println("\nU=\r")
U


U=


6-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
 -4.761904761904763
 14.28571428571429

In [14]:
#computing the global nodal force vector
F=K*U
println("\nF=\r")
F


F=


6-element Array{Float64,1}:
 -10000.000000000002
 -10000.0
  10000.000000000002
      0.0
      1.8189894035458565e-12
  10000.0

# Calcul et Affichage des grandeurs dérivées déformations et contraintes élémentaires

In [15]:
#COMPUTING STRESSES
#writing the element nodal displacement vectors
U1=[U[1] U[2] U[5] U[6]]'
U2=[U[3] U[4] U[5] U[6]]'
#computing element strains
ϵ1=D2_TrussElementStrain(L1,45,U1)
println("\nϵ1=\r")
ϵ1


ϵ1=


1×1 Array{Float64,2}:
 0.00047619047619047624

In [16]:
ϵ2=D2_TrussElementStrain(L2,0,U2)
println("\nϵ2=\r")
ϵ2


ϵ2=


1×1 Array{Float64,2}:
 -0.0004761904761904763

In [17]:
#computing element forces
f1=D2_TrussElementForce(E1,A1,L1,45,U1)
f2=D2_TrussElementForce(E2,A2,L2,0,U2)
#computing element stresses
σ1=D2_TrussElementStress(E1,L1,45,U1)
println("\nσ1=\r")
σ1


σ1=


1×1 Array{Float64,2}:
 100.00000000000004

In [18]:
σ2=D2_TrussElementStress(E2,L2,0,U2)
println("\nσ2=\r")
σ2


σ2=


1×1 Array{Float64,2}:
 -100.00000000000001